In [28]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

project_root = Path("/dmj/fizmed/mmarzec/licencjat_neuro/magisterka")
sys.path.insert(0, str(project_root / "src"))

In [29]:
from preprocessing.frame_exporter import EEGFrameExporter #type:ignore
from connectivity.mtmvar import DTF_multivariate, mvar_plot #type:ignore
from preprocessing.constants import DEFAULT_HP_CUTOFF, DEFAULT_LP_CUTOFF #type:ignore

In [30]:
groups = EEGFrameExporter.load_all_groups("demoDTF/1/frames")
info = EEGFrameExporter.load_group_info("demoDTF/1/frames")

print(f"Wczytano {len(groups)} grup:")
for name in groups.keys():
    print(f"  • {name}: {groups[name].shape[0]} ramek")

Wczytano 8 grup:
  • eyes_closed: 3 ramek
  • eyes_opened: 3 ramek
  • stimulation_hyperventilationstart: 19 ramek
  • stimulation_hyperventilationstop: 1 ramek
  • stimulation_photostimulationstart: 1 ramek
  • stimulation_photostimulationstop: 1 ramek
  • technical_exampause: 2 ramek
  • technical_examresume: 2 ramek


In [31]:
MAX_P = 20
P_OPT = None  # Automatyczny wybór
CRIT_TYPE = 'AIC'
FMIN = DEFAULT_HP_CUTOFF
FMAX = DEFAULT_LP_CUTOFF

# Wektor częstotliwości
f = np.arange(1,30, 0.1)

print(f"\nParametry DTF:")
print(f"  Frequency: {DEFAULT_HP_CUTOFF}-{DEFAULT_LP_CUTOFF} Hz")
print(f"  Max model order: {MAX_P}")
print(f"  Criterion: {CRIT_TYPE}")


Parametry DTF:
  Frequency: 1.0-40.0 Hz
  Max model order: 20
  Criterion: AIC


EYES OPENED - DTF dla każdej ramki

In [32]:
group_name = 'eyes_opened'
frames_eo = groups[group_name]
sfreq_eo = info[group_name]['sfreq']
channels_eo = info[group_name]['channel_names']

print(f"\nGrupa: {group_name}")
print(f"  Liczba ramek: {len(frames_eo)}")
print(f"  Sampling rate: {sfreq_eo} Hz")

# Katalog do zapisywania wykresów
output_dir_eo = Path("demoDTF/1/dtf_eyes_opened")
output_dir_eo.mkdir(exist_ok=True, parents=True)

# Oblicz DTF dla każdej ramki
for i, frame in enumerate(frames_eo):
    print(f"\nRamka {i+1}/{len(frames_eo)}")
    print(f"  Shape: {frame.shape}")
    
    # Oblicz DTF
    dtf = DTF_multivariate(
        signals=frame,
        f=f,
        Fs=sfreq_eo,
        max_p=MAX_P,
        p_opt=P_OPT,
        crit_type=CRIT_TYPE
    )
    
    
    # Wizualizacja
    fig = mvar_plot(
        onDiag=dtf,
        offDiag=dtf,
        f=f,
        xlab='Frequency [Hz] → ',
        ylab='← ',
        ChanNames=channels_eo,
        Top_title=f'DTF - Eyes Opened - Frame {i+1}/{len(frames_eo)}',
        scale='linear'
    )
    
    # Zapisz wykres
    output_file = output_dir_eo / f"dtf_eo_frame_{i+1:03d}.png"
    plt.savefig(output_file, dpi=150, bbox_inches='tight')
    print(f"  ✓ Zapisano: {output_file.name}")
    
    plt.close()  # Zamknij figure aby zwolnić pamięć

print(f"\n✓ Wszystkie wykresy Eyes Opened zapisane w: {output_dir_eo}")


Grupa: eyes_opened
  Liczba ramek: 3
  Sampling rate: 128.0 Hz

Ramka 1/3
  Shape: (19, 768)
Optimal model order for all channels: p =  1


/dmj/fizmed/mmarzec/licencjat_neuro/magisterka/src/connectivity/mtmvar.py:726: RuntimeWarning: invalid value encountered in log
  crit[p-1] = np.log(np.linalg.det(Vr)) + 2 * p * k**2 / N


  ✓ Zapisano: dtf_eo_frame_001.png

Ramka 2/3
  Shape: (19, 768)
Optimal model order for all channels: p =  1


/dmj/fizmed/mmarzec/licencjat_neuro/magisterka/src/connectivity/mtmvar.py:726: RuntimeWarning: invalid value encountered in log
  crit[p-1] = np.log(np.linalg.det(Vr)) + 2 * p * k**2 / N


KeyboardInterrupt: 